## **Importando pacotes**

In [5]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import sys


from sklearn.metrics 	     import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.compose 	     import ColumnTransformer
from sklearn.pipeline 	     import Pipeline
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.preprocessing   import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.impute          import KNNImputer

from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import SVC
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost                 import XGBClassifier
from lightgbm                import LGBMClassifier
from catboost                import CatBoostClassifier
from sklearn.dummy           import DummyClassifier
from sklearn.neighbors       import KNeighborsClassifier

from sklearn.feature_selection import SelectKBest
from category_encoders.target_encoder import TargetEncoder


sys.path.append('../utils')

from modelcrafterclass import ModelCrafter

## **Carregando os dados**

In [6]:
data = pd.read_csv('../datasets_for_ml/dataset_for_train.csv')

In [7]:
data.head()

,Client,data_recente,target,qt_faltas,moda_staff_faltante,moda_servico_faltante,moda_servico_cancelado,moda_staff_cancelado,antecedencia,qt_cancelamentos,moda_staff_prestou_servico,moda_dia,qte_servicos_por_dia,media,mediana,desvio_padrao,min,max,qte_servico_recebido
0,KERT01,2018-06-20,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,JJ,Tuesday,1.5,84.666667,82.0,16.165808,70.0,102.0,3
1,COOM01,2018-06-15,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,SINEAD,Thursday,1.0,70.000000,70.0,NaN,70.0,70.0,1
2,PEDM01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,BECKY,Saturday,1.0,60.000000,60.0,NaN,60.0,60.0,1
3,BAIS01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0
4,FRAL01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0


Apesar de haver valores faltantes, esses não são advindos de erros ou problemas. Na realidade, esses valores nulos tem seus significados dentro do contexto do nosso problema.

Por exemplo, um registro com a media faltante está associado a um novo cliente. Antecedência faltante indica que um cliente nunca fez um cancelamento e assim por diante.

Portanto, devemos achar uma maneira de introduzir essas informações na nossa base de dados

In [15]:
data.isna().sum().to_frame('qte. missing').sort_values(by='qte. missing',ascending=False)

,qte. missing
antecedencia,734
desvio_padrao,590
max,461
min,461
mediana,461
media,461
qte_servicos_por_dia,461
Client,0
moda_staff_prestou_servico,0
moda_dia,0


Como todos os valores nulos são numéricos, vamos substitui-los por -1

In [25]:
data = data.fillna(-1)

Vamos adicionar uma nova coluna, indicando se temos um novo cliente. Para fazer isso vamos tomar como base a variável média (poderia ser qualquer outra). Todas as instâncias com media faltante receberão 1 e as demais 0

In [27]:
data['novo_cliente'] = data.apply(lambda x: 1 if x['media'] == -1 else 0,axis=1)

# **Breve EDA**

Ao todo há 798 registros, ou seja, 798 clientes únicos

In [30]:
data.shape

(798, 20)

Estamos lidando com um problema de classes desbalanceadas onde 751 (94%) clientes seguiram as políticas do salão enquanto 47 (5.9%) não seguiram

In [43]:
pd.concat([data['target'].value_counts().to_frame('Absoluto'),round(data['target'].value_counts(normalize=True).mul(100).to_frame('%'),2)],axis=1)

,Absoluto,%
target,,
0,751,94.11
1,47,5.89


Observamos que 461 (58%) são novos clientes e 337 (42%) são clientes com algum histórico.

In [45]:
pd.concat([data['novo_cliente'].value_counts().to_frame('Absoluto'),round(data['novo_cliente'].value_counts(normalize=True).mul(100).to_frame('%'),2)],axis=1)

,Absoluto,%
novo_cliente,,
1,461,57.77
0,337,42.23


Da observação acima, podemos notar um problema. Mais da metade do dataset são novos cliente e, portanto, não tem nenhum histórico associado.

Clientes sem histórico podem ser um problema, pois a única informação que adquirimos sobre eles está no momento da reserva, o que pode não ser muito informativo.

> Para lidar com isso, nós vamos treinar um modelo com todos os clientes (novos e antigos) e retreinar esses modelos para apenas clientes antigos. E avaliar os resultados para tomarmos uma decisão.